In [2]:
import re
import os
import glob
import json
import pytesseract
from PIL import Image

In [2]:
row_height = 590
rows_start = 330
left_margin_rows = 220
right_margin_rows = 245
box_width = 1500

In [3]:
def get_string_from_row(row):
    '''Takes in a single row (Image Object), gets 3 text strings out'''
    return [pytesseract.image_to_string(row.crop((i * box_width, 0 ,(i + 1) * box_width, row_height))) for i in range(3)]

In [4]:
regexes = {
    'Name': "(?<=Name)(.*)",
    'Relative': "(Father|Mother|Husband)\\\'s.*?Name.*?:(.*)",
    'Age': "Age:\s([0-9]+)",
    'Sex': "Sex:\s(MALE|FEMALE)"
}
compiled_regexes = {k:re.compile(v) for k,v in regexes.items()}
def handle_sheet(filename):
    '''Takes a filename and returns a list of strings inside it'''
    image = Image.open('./142_page_18.png')
    width, height = image.size
    header_box = (0,0,width, 320)
    footer_box = (0, 6220, width, height)
    rows = [image.crop((left_margin_rows, rows_start+ (i-1)*row_height, width-right_margin_rows, rows_start+ i*row_height)) for i in range(1, 11)]
    strings = [get_string_from_row(row) for row in rows]
    all_strings = [item for sublist in strings for item in sublist]
    regexes = [[regex.search(string_)[0] if regex.search(string_) else None for regex in compiled_regexes.values()] for string_ in all_strings]
    return{
        'Header': pytesseract.image_to_string(image.crop(header_box)),
        'Footer': pytesseract.image_to_string(image.crop(footer_box)),
#         'Rows': rows,
        'Strings': strings,
        "Regexes": regexes
    }

In [ ]:
'''This is a dumb, lazy approach'''
num_files = sorted([int(re.search('[0-9]+', filename)[0]) for filename in glob.glob('./*.png')])
usable_files = [f'page_{num}.png' for num in num_files[2:-1]]
for file in usable_files:
    with open(file+'.json', 'w') as f:
        json.dump(handle_sheet(file), f)

In [5]:
jsons = glob.glob('./*.json')

In [6]:
jsons


['./page_38.png.json',
 './page_21.png.json',
 './page_15.png.json',
 './page_11.png.json',
 './page_34.png.json',
 './page_36.png.json',
 './page_29.png.json',
 './page_8.png.json',
 './page_32.png.json',
 './page_35.png.json',
 './page_22.png.json',
 './page_27.png.json',
 './page_43.png.json',
 './page_40.png.json',
 './page_41.png.json',
 './page_39.png.json',
 './page_10.png.json',
 './page_5.png.json',
 './page_3.png.json',
 './page_44.png.json',
 './page_23.png.json',
 './page_25.png.json',
 './page_30.png.json',
 './page_6.png.json',
 './page_16.png.json',
 './page_28.png.json',
 './page_13.png.json',
 './page_17.png.json',
 './page_37.png.json',
 './page_20.png.json',
 './page_33.png.json',
 './page_31.png.json',
 './page_26.png.json',
 './page_9.png.json',
 './page_46.png.json',
 './page_4.png.json',
 './page_19.png.json',
 './page_18.png.json',
 './page_24.png.json',
 './page_45.png.json',
 './page_7.png.json',
 './page_42.png.json',
 './page_14.png.json',
 './page_12.png.js

In [6]:
with open('./page_3.png.json')as f:
    x = json.load(f)

In [9]:
num_files = sorted([int(re.search('[0-9]+', filename)[0]) for filename in glob.glob('./*.json')])
dataframes = []
for file in num_files:
    with open(f'./page_{file}.png.json') as f:
        x = json.load(f)
        dataframes += [pd.DataFrame(x['Regexes'])]

In [8]:
num_files = sorted([int(re.search('[0-9]+', filename)[0]) for filename in glob.glob('./*.json')])
import pandas as pd

In [15]:
df = pd.concat(dataframes).dropna(how='all')
df.columns = ['Name', 'Relative', 'Age', 'Sex']
df['Name'] = df['Name'].apply(lambda x: x.split(':')[-1].strip() if isinstance(x, str) else None)
df['Age'] = df['Age'].apply(lambda x: x.split(':')[-1].strip() if isinstance(x, str) else None)
df['Sex'] = df['Sex'].apply(lambda x: x.split(':')[-1].strip() if isinstance(x, str) else None)


In [17]:
df.to_csv('./first_pdf.csv', index=None)